<a href="https://colab.research.google.com/github/Vaibhavs10/notebooks/blob/main/use_encodec_w_%F0%9F%A4%97transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Encodec is now in 🤗 Transformers

Want to train your own Bark, MusicGen like models using SoTA audio codebook embeddings powered by Encodec?

Look no more! Try it out today, all in less than 10 lines of code ♥

### Set up our colab's development environment!

We only need `datasets` and `transformers`. Since Encodec was merged after the last release, we'll install transformers from source.

In [13]:
!pip install -q datasets git+https://github.com/huggingface/transformers.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import the datasets library and encodec model from transformers

In [2]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor

For the purposes of this demonstration, we'll leverage a dummy dataset, however, you can swap this for any dataset on the hub or bring your own.

In [14]:
# dummy dataset, however you can swap this with an dataset on the 🤗 hub or bring your own
librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# load the model + processor (for pre-processing the audio)
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


First, we'd need to pre-process the audio with the sampling rate needed for the model!

In [4]:
# cast the audio data to the correct sampling rate for the model
librispeech_dummy = librispeech_dummy.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
audio_sample = librispeech_dummy[0]["audio"]["array"]

Let's give this audio file a listen!

In [19]:
import IPython.display as ipd

ipd.Audio(audio_sample, rate=processor.sampling_rate)

In [5]:
# pre-process the audio inputs
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

To compress the audio and get the model output waveform we can extract the output from the `model.decode`

In [20]:
# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])

# pass the encoder outputs to the decoder to get the compressed waveform
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]

Since we did not specify a target `bandwidth` by default the model compresses the audio to 1.5KHz (the lowest possible compression rate.

You can change this by passing in `bandwidth` along with the `model.encode` call.

```python
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"], bandwidth = 24.0)
```

The model supports: `1.5`, `3.0`, `6.0`, `12.0`, `24.0`Khz bandwidth.

That's it and now let's compare the compressed waveform.

In [30]:
import IPython.display as ipd

ipd.Audio(audio_values.detach().numpy()[0][0], rate=processor.sampling_rate)

Bonus: You can use Encodec to extract discrete codebook representation for your input audio!

These discrete representations can then be used for downstream audio language modeling tasks like Text-to-Speech, Text-to-Music and so on.

P.S. [Bark](https://github.com/suno-ai/bark) and [MusicGen](https://github.com/facebookresearch/audiocraft) also use Encodec under the hood! ⚡️

In [9]:
audio_codes = model(inputs["input_values"], inputs["padding_mask"]).audio_codes

In [31]:
audio_codes

tensor([[[[  62,  835,  835,  835,  835,  835,  835,  835,  408,  408,  835,
            835,  835,  835,  835,  408,  408,  408,  408,  408,  408,  408,
            408,  408,  408,  408,  408,  408,  408,  408,  408,  408,  408,
            408,  408,  408,  408,  408,  408,  408,  339,  228,  570,  991,
            681,  972,  969,  303,   38,  463,  738,  106,  855,  602,  142,
            511,  722,  860,  604,  876,  738,  106, 1014,  405,  405,  488,
            461,  461,  461,  293,  736,  933,  894,  723,  784,  837,  291,
           1000,   52, 1019,  488,  854,  872,  585,  991,  784,  723,   11,
            722,  722,  722,  681,  723,  723,  723,  681,  734,  825,  534,
            972,  303,   53,   53,  463,  463,   53,  373,  373,  136,  991,
            840,  765,  407,  303,  840,  916,  613,  393,  430,  876,  738,
            865,  408,  738,  904,   40,  103,  731,  731,  935,  935,   80,
             20,  953,  593,   99,  132,  457,  254,  899,   53,   53,  463,

In [11]:
audio_codes.shape

torch.Size([1, 1, 2, 440])